In [1]:
#3.3 Concise Implementation of Linear Regression
#3.2是用自己定義的function, 3.3是直接用套件的api

import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

#3.3.1 Generating the Dataset

def synthetic_data(w, b, num_examples):
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w))) 
    # torch.normal(means, std, out=None)從mean跟std的離散正態分佈中抽取隨機數
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4]) 
#每個example有2個features(independent variable)
#因此data set會是1000*2的matrix

true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000) #產生1000個examples
 

In [15]:
#3.3.2 Reading the Dataset

def load_array(data_arrays, batch_size, is_train=True): #is_train:是否希望每跑一次就打亂數據一次
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [18]:
print(next(iter(data_iter))) #用next獲取項目
print(next(iter(data_iter)))

[tensor([[-0.0158, -1.7598],
        [-1.1121, -1.6656],
        [ 1.7333,  1.0287],
        [ 1.5167,  0.2330],
        [ 0.1456,  1.2308],
        [ 0.4771, -1.1848],
        [-1.0472, -1.4159],
        [ 0.5294, -0.8551],
        [-1.5126, -0.2875],
        [-0.2788, -0.3877]]), tensor([[10.1390],
        [ 7.6473],
        [ 4.1709],
        [ 6.4505],
        [ 0.2964],
        [ 9.1931],
        [ 6.8982],
        [ 8.1663],
        [ 2.1726],
        [ 4.9627]])]
[tensor([[ 0.2540,  1.3479],
        [ 0.7311,  0.8951],
        [-1.1121, -1.6656],
        [-0.0827, -0.3859],
        [ 0.3451,  0.0022],
        [ 0.5348, -0.2349],
        [ 0.5509, -1.3352],
        [-0.0316, -0.0986],
        [-0.1609,  1.8770],
        [ 1.7217,  0.4935]]), tensor([[ 0.1351],
        [ 2.6002],
        [ 7.6473],
        [ 5.3458],
        [ 4.8910],
        [ 6.0582],
        [ 9.8453],
        [ 4.4640],
        [-2.4979],
        [ 5.9766]])]


In [23]:
#3.3.3 Defining the Model

# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))
print(net)
print(net[0])


Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [25]:
#3.3.4 Initializing Model Parameters

net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [26]:
#3.3.5 Defining the Loss Function

loss = nn.MSELoss()

In [27]:
#3.3.6 Defining the Optimization Algorithm

trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [32]:
#3.3.7 Training

num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000101
epoch 2, loss 0.000101
epoch 3, loss 0.000104


In [33]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([-0.0002, -0.0020])
error in estimating b: tensor([0.0006])
